In [ ]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 28.1 MB/s 
     |████████████████████████████████| 1.6 MB 59.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [ ]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(8,)))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 3s 11ms/step - loss: 0.7068 - accuracy: 0.5065 - val_loss: 0.6702 - val_accuracy: 0.6494
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6510 - accuracy: 0.6482 - val_loss: 0.6223 - val_accuracy: 0.7143
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6093 - accuracy: 0.7020 - val_loss: 0.5871 - val_accuracy: 0.7727
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5789 - accuracy: 0.7166 - val_loss: 0.5599 - val_accuracy: 0.7727
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5542 - accuracy: 0.7313 - val_loss: 0.5389 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5348 - accuracy: 0.7394 - val_loss: 0.5243 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5201 - accuracy: 0.7427 - val_loss: 0.5114 - val_accuracy: 0.7857
Epoch 8/100


#select appropriate optimizer
#no.of nodes in a layer
#no. of hidden layers
#all in one

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_shape=(8,)))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop",'adadelta','adagrad'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=["accuracy"])
  return model

In [ ]:
tuner=kt.RandomSearch(build_model,
                      objective="val_accuracy",
                      max_trials=5,
                      directory="tuning",
                      project_name="version1")

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.3636363744735718

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 06s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5236 - accuracy: 0.7492 - val_loss: 0.5240 - val_accuracy: 0.7987
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5120 - accuracy: 0.7492 - val_loss: 0.5139 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5027 - accuracy: 0.7557 - val_loss: 0.5054 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4946 - accuracy: 0.7655 - val_loss: 0.4989 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4884 - accuracy: 0.7671 - val_loss: 0.4923 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4828 - accuracy: 0.7687 - val_loss: 0.4858 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4780 - accuracy: 0.7720 - val_loss: 0.4817 - val_accuracy: 0.7922
Epoch 13/

No. of neurons

In [ ]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units',8,128,1)
  model.add(Dense(units=units,activation='relu',input_shape=(8,)))
  model.add(Dense(units=1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                   max_trials=3,
                      directory='tuning',
                      project_name='version2')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 05s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 68}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 10ms/step - loss: 0.4856 - accuracy: 0.7720 - val_loss: 0.4758 - val_accuracy: 0.8117
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4761 - accuracy: 0.7769 - val_loss: 0.4697 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4697 - accuracy: 0.7720 - val_loss: 0.4642 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4649 - accuracy: 0.7704 - val_loss: 0.4590 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4614 - accuracy: 0.7671 - val_loss: 0.4558 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4584 - accuracy: 0.7687 - val_loss: 0.4559 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4559 - accuracy: 0.7736 - val_loss: 0.4568 - val_accuracy: 0.7857
Epoch 13/

#No. of hidden layers

In [ ]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):# i in range (4)
    model.add(Dense(72,activation='relu'))
  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
tuner=kt.RandomSearch(build_model,
                      objective="val_accuracy",
                      max_trials=3,
                      directory='tuning',
                      project_name='version3')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 07s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 72)                5256      
                                                                 
 dense_6 (Dense)             (None, 72)                5

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 13ms/step - loss: 0.4543 - accuracy: 0.7834 - val_loss: 0.4731 - val_accuracy: 0.8247
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4406 - accuracy: 0.7850 - val_loss: 0.5439 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4336 - accuracy: 0.8062 - val_loss: 0.5015 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4340 - accuracy: 0.8029 - val_loss: 0.4592 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4182 - accuracy: 0.8094 - val_loss: 0.4754 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4054 - accuracy: 0.8094 - val_loss: 0.5014 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3920 - accuracy: 0.8208 - val_loss: 0.4952 - val_accuracy: 0.7922
Epoch 13/

#All in one

In [ ]:
[i/10 for i in range(1,10)]

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
def build_model(hp):
  model=Sequential()
  counter=0
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    if counter==0:
      model.add(
          Dense(
              units=hp.Int("units"+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('Activation'+str(i),values=['relu','tanh','sigmoid']),
              input_dim=8)
      )
      
    else:
      model.add(
          Dense(
              units=hp.Int("units"+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('Activation'+str(i),values=['relu','tanh','sigmoid']),
              )
      )
    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice("optimizer",values=['adam','rmsprop','adadelta','sgd','nadam'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [ ]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='tuning',project_name='version4')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 05s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 120,
 'Activation0': 'sigmoid',
 'optimizer': 'rmsprop',
 'units1': 72,
 'Activation1': 'relu',
 'units2': 16,
 'Activation2': 'sigmoid',
 'units3': 8,
 'Activation3': 'relu'}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 72)                8712      
                                                                 
 dense_2 (Dense)             (None, 16)                1168      
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 11,105
Trainable params: 11,105
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 1s 13ms/step - loss: 0.5558 - accuracy: 0.7280 - val_loss: 0.5385 - val_accuracy: 0.7208
Epoch 7/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5381 - accuracy: 0.7362 - val_loss: 0.5209 - val_accuracy: 0.7468
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5212 - accuracy: 0.7427 - val_loss: 0.5140 - val_accuracy: 0.7532
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5127 - accuracy: 0.7638 - val_loss: 0.4934 - val_accuracy: 0.7792
Epoch 10/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5022 - accuracy: 0.7655 - val_loss: 0.4879 - val_accuracy: 0.7987
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4927 - accuracy: 0.7671 - val_loss: 0.4878 - val_accuracy: 0.7662
Epoch 12/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4892 - accuracy: 0.7785 - val_loss: 0.4871 - val_accuracy: 0.8117
Epoch 13/

In [ ]:
def build_model(hp):
  model=Sequential()
  counter=0
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(units=hp.Int("units"+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('Activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice("dropout"+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))) 
    else:
      model.add(Dense(units=hp.Int("units"+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('Activation'+str(i),values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice("dropout"+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice("optimizer",values=['adam','rmsprop','adadelta','sgd','nadam'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [ ]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=3,directory="tuning",project_name="final")

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 56,
 'Activation0': 'tanh',
 'dropout0': 0.4,
 'optimizer': 'nadam',
 'units1': 8,
 'Activation1': 'tanh',
 'dropout1': 0.3,
 'units2': 24,
 'Activation2': 'relu',
 'dropout2': 0.9,
 'units3': 120,
 'Activation3': 'tanh',
 'dropout3': 0.2,
 'units4': 112,
 'Activation4': 'tanh',
 'dropout4': 0.2,
 'units5': 16,
 'Activation5': 'tanh',
 'dropout5': 0.1,
 'units6': 8,
 'Activation6': 'relu',
 'dropout6': 0.1}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                504       
                                                                 
 dropout (Dropout)           (None, 56)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 456       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 24)                216       
                                                                 
 dropout_2 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 120)               3

In [ ]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 2s 16ms/step - loss: 0.6377 - accuracy: 0.6710 - val_loss: 0.5423 - val_accuracy: 0.7403
Epoch 7/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6326 - accuracy: 0.6515 - val_loss: 0.5398 - val_accuracy: 0.7532
Epoch 8/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6320 - accuracy: 0.6596 - val_loss: 0.5418 - val_accuracy: 0.7532
Epoch 9/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6442 - accuracy: 0.6564 - val_loss: 0.5485 - val_accuracy: 0.7338
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6250 - accuracy: 0.6678 - val_loss: 0.5435 - val_accuracy: 0.7273
Epoch 11/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6111 - accuracy: 0.6694 - val_loss: 0.5300 - val_accuracy: 0.7597
Epoch 12/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6312 - accuracy: 0.6645 - val_loss: 0.5173 - val_accuracy: 0.7662
Epoch 13/